In [ ]:
import requests
import pandas as pd

# Replace 'YOUR_API_KEY' with your actual Census API key


# Specify the dataset and variables
dataset = 'acs/acs5'  # American Community Survey 5-Year Estimates
get_vars = 'NAME,B01001_001E'  # Variable for total population (B01001_001E)

# California's FIPS code
for_state = '06'

# Years range from 2015 to 2022
years = range(2015, 2024)

# Create an empty list to store the data
data_list = []

for year in years:
    # Make the API request
    base_url = f'https://api.census.gov/data/{year}/{dataset}?get={get_vars}&for=place:*&in=state:{for_state}&key={api_key}'
    response = requests.get(base_url)

    if response.status_code == 200:
        data = response.json()
        # Add retrieved data to the list
        for entry in data[1:]:  # Skip the header row
            city_name = entry[0]
            population = entry[1]
            data_list.append({'City': city_name, 'Population': population, 'Year': year})
    else:
        print(f"Failed to retrieve data for year {year}.")

# Create a DataFrame from the collected data
population_data = pd.DataFrame(data_list)

# Display the DataFrame
print(population_data)